In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys

In [13]:
class DCGAN():
    def __init__(self, rows, cols, channels, z = 10):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z
        
        optimizer = tf.keras.optimizers.Adam(lr = 0.0002, beta_1 = 0.5)
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
                                   optimizer = optimizer,
                                   metrics = ['accuracy'])
        
        # Build the generator
        self.generator = self.build_generator()
        
        # The generator takes noise as input and generates imgs
        z = tf.keras.layers.Input(shape = (self.latent_dim,))
        img = self.generator(z)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)
        
        # The combined model (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = tf.keras.models.Model(z, valid)
        self.combined.compile(loss = 'binary_crossentropy',
                              optimizer = optimizer)
        
    def build_generator(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(128 * 7 * 7, activation = 'relu', input_dim = self.latent_dim),
            tf.keras.layers.Reshape((7, 7, 128)),
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.Activation(activation='relu'),
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.Activation(activation = 'relu'),
            tf.keras.layers.Conv2D(self.channels, kernel_size = 3, padding = 'same'),
            tf.keras.layers.Activation(activation = 'tanh')
        ])
        model.summary()
        noise = tf.keras.layers.Input(shape = (self.latent_dim,))
        img = model(noise)
        return tf.keras.models.Model(noise, img)
    
    def build_discriminator(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 2,
                                   input_shape = self.img_shape, padding = 'same'),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same'),
            tf.keras.layers.ZeroPadding2D(padding = ((0,1), (0,1))),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 2, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 2, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
        ])
        model.summary()
        img = tf.keras.layers.Input(shape = (self.img_shape))
        validity = model(img)
        return tf.keras.models.Model(img, validity)
    
    def train(self, epochs, batch_size = 256, save_interval = 50):
        # Load the dataset
        (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
        
        # Rescale -1 to 1
        x_train = x_train / 127.5 - 1.
        x_train = np.expand_dims(x_train, axis = 3)
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            # -------------------
            # Train Discriminator
            # -------------------
            
            # Select a random half of images
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs = x_train[idx]
            
            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # -------------------
            # Train Generator
            # -------------------
            
            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
    
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap = 'gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig('dcgan_mnist_%d.png' % epoch)
        plt.close()

In [ ]:
dcgan = DCGAN(28, 28, 1)
dcgan.train(epochs = 5000, batch_size = 256, save_interval = 50)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 8, 8, 64)         

65 [D loss: 0.757593, acc.: 51.37%] [G loss: 0.871541]
66 [D loss: 0.842531, acc.: 47.27%] [G loss: 0.870587]
67 [D loss: 0.848778, acc.: 43.16%] [G loss: 0.874184]
68 [D loss: 0.875641, acc.: 43.16%] [G loss: 0.975166]
69 [D loss: 0.834733, acc.: 46.09%] [G loss: 1.029558]
70 [D loss: 0.905060, acc.: 39.45%] [G loss: 1.003478]
71 [D loss: 0.830799, acc.: 44.34%] [G loss: 1.000226]
72 [D loss: 0.787187, acc.: 45.70%] [G loss: 1.000954]
73 [D loss: 0.808638, acc.: 49.80%] [G loss: 0.956417]
74 [D loss: 0.816821, acc.: 44.34%] [G loss: 0.958984]
75 [D loss: 0.812170, acc.: 46.48%] [G loss: 0.947565]
76 [D loss: 0.786851, acc.: 50.20%] [G loss: 0.908335]
77 [D loss: 0.761651, acc.: 50.98%] [G loss: 0.881955]
78 [D loss: 0.759645, acc.: 50.20%] [G loss: 0.894150]
79 [D loss: 0.704669, acc.: 56.05%] [G loss: 0.884837]
80 [D loss: 0.651372, acc.: 61.91%] [G loss: 0.946395]
81 [D loss: 0.650115, acc.: 61.91%] [G loss: 0.911990]
82 [D loss: 0.624300, acc.: 65.04%] [G loss: 0.977575]
83 [D loss

212 [D loss: 0.692478, acc.: 55.86%] [G loss: 0.844776]
213 [D loss: 0.727448, acc.: 53.52%] [G loss: 0.895616]
214 [D loss: 0.889699, acc.: 35.16%] [G loss: 0.862720]
215 [D loss: 0.785526, acc.: 46.48%] [G loss: 0.846781]
216 [D loss: 0.663310, acc.: 59.18%] [G loss: 0.849362]
217 [D loss: 0.608727, acc.: 67.58%] [G loss: 0.821944]
218 [D loss: 0.553895, acc.: 73.44%] [G loss: 0.791874]
219 [D loss: 0.623850, acc.: 65.04%] [G loss: 0.799788]
220 [D loss: 0.618035, acc.: 67.97%] [G loss: 0.807028]
221 [D loss: 0.590245, acc.: 67.97%] [G loss: 0.800010]
222 [D loss: 0.629162, acc.: 66.41%] [G loss: 0.793690]
223 [D loss: 0.680603, acc.: 57.81%] [G loss: 0.786596]
224 [D loss: 0.655612, acc.: 59.57%] [G loss: 0.797794]
225 [D loss: 0.708729, acc.: 54.69%] [G loss: 0.835113]
226 [D loss: 0.736557, acc.: 53.12%] [G loss: 0.824066]
227 [D loss: 0.574479, acc.: 71.29%] [G loss: 0.813830]
228 [D loss: 0.723754, acc.: 54.88%] [G loss: 0.835706]
229 [D loss: 0.737123, acc.: 52.93%] [G loss: 0.

359 [D loss: 1.072341, acc.: 23.83%] [G loss: 0.994143]
360 [D loss: 0.823974, acc.: 45.51%] [G loss: 0.967679]
361 [D loss: 0.432615, acc.: 80.66%] [G loss: 0.982631]
362 [D loss: 0.309859, acc.: 93.16%] [G loss: 0.860422]
363 [D loss: 0.199754, acc.: 98.44%] [G loss: 0.839909]
364 [D loss: 0.157603, acc.: 99.80%] [G loss: 0.733836]
365 [D loss: 0.145067, acc.: 99.22%] [G loss: 0.764777]
366 [D loss: 0.294508, acc.: 92.19%] [G loss: 0.733734]
367 [D loss: 0.487293, acc.: 76.17%] [G loss: 0.926472]
368 [D loss: 0.759585, acc.: 50.98%] [G loss: 1.086080]
369 [D loss: 1.062054, acc.: 28.52%] [G loss: 0.881014]
370 [D loss: 0.937012, acc.: 31.84%] [G loss: 0.713024]
371 [D loss: 0.535809, acc.: 72.66%] [G loss: 0.652342]
372 [D loss: 0.187653, acc.: 96.48%] [G loss: 0.729933]
373 [D loss: 0.099134, acc.: 99.22%] [G loss: 0.775463]
374 [D loss: 0.052518, acc.: 100.00%] [G loss: 0.825275]
375 [D loss: 0.046011, acc.: 100.00%] [G loss: 0.778253]
376 [D loss: 0.045830, acc.: 100.00%] [G loss:

505 [D loss: 0.016861, acc.: 100.00%] [G loss: 0.703922]
506 [D loss: 0.019362, acc.: 100.00%] [G loss: 0.675991]
507 [D loss: 0.016041, acc.: 100.00%] [G loss: 0.672417]
508 [D loss: 0.020469, acc.: 100.00%] [G loss: 0.739646]
509 [D loss: 0.020679, acc.: 100.00%] [G loss: 0.750411]
510 [D loss: 0.019274, acc.: 100.00%] [G loss: 0.781330]
511 [D loss: 0.031483, acc.: 100.00%] [G loss: 0.737166]
512 [D loss: 0.024348, acc.: 100.00%] [G loss: 0.791709]
513 [D loss: 0.046463, acc.: 100.00%] [G loss: 0.861051]
514 [D loss: 0.050092, acc.: 99.80%] [G loss: 0.882816]
515 [D loss: 0.077187, acc.: 100.00%] [G loss: 0.924126]
516 [D loss: 0.160563, acc.: 98.44%] [G loss: 1.042517]
517 [D loss: 0.171519, acc.: 99.02%] [G loss: 1.066496]
518 [D loss: 0.210116, acc.: 95.51%] [G loss: 0.920026]
519 [D loss: 0.253373, acc.: 94.73%] [G loss: 0.761351]
520 [D loss: 0.244682, acc.: 94.73%] [G loss: 0.754157]
521 [D loss: 0.095893, acc.: 99.61%] [G loss: 0.861948]
522 [D loss: 0.050905, acc.: 100.00%] 

...

2501 [D loss: 0.000097, acc.: 100.00%] [G loss: 0.013101]
2502 [D loss: 0.000109, acc.: 100.00%] [G loss: 0.013500]
2503 [D loss: 0.000079, acc.: 100.00%] [G loss: 0.012948]
2504 [D loss: 0.000083, acc.: 100.00%] [G loss: 0.014480]
2505 [D loss: 0.000083, acc.: 100.00%] [G loss: 0.013524]
2506 [D loss: 0.000076, acc.: 100.00%] [G loss: 0.010413]
2507 [D loss: 0.000096, acc.: 100.00%] [G loss: 0.014725]
2508 [D loss: 0.000080, acc.: 100.00%] [G loss: 0.011291]
2509 [D loss: 0.000109, acc.: 100.00%] [G loss: 0.013421]
2510 [D loss: 0.000096, acc.: 100.00%] [G loss: 0.013522]
2511 [D loss: 0.000099, acc.: 100.00%] [G loss: 0.012236]
2512 [D loss: 0.000090, acc.: 100.00%] [G loss: 0.014340]
2513 [D loss: 0.000195, acc.: 100.00%] [G loss: 0.015227]
2514 [D loss: 0.000112, acc.: 100.00%] [G loss: 0.013206]
2515 [D loss: 0.000128, acc.: 100.00%] [G loss: 0.013939]
2516 [D loss: 0.000093, acc.: 100.00%] [G loss: 0.012225]
2517 [D loss: 0.000093, acc.: 100.00%] [G loss: 0.012155]
2518 [D loss: 

2643 [D loss: 0.000061, acc.: 100.00%] [G loss: 0.008052]
2644 [D loss: 0.000135, acc.: 100.00%] [G loss: 0.009943]
2645 [D loss: 0.000055, acc.: 100.00%] [G loss: 0.008656]
2646 [D loss: 0.000076, acc.: 100.00%] [G loss: 0.009102]
2647 [D loss: 0.000063, acc.: 100.00%] [G loss: 0.008484]
2648 [D loss: 0.000094, acc.: 100.00%] [G loss: 0.006880]
2649 [D loss: 0.000079, acc.: 100.00%] [G loss: 0.008748]
2650 [D loss: 0.000079, acc.: 100.00%] [G loss: 0.008500]
2651 [D loss: 0.000079, acc.: 100.00%] [G loss: 0.008710]
2652 [D loss: 0.000101, acc.: 100.00%] [G loss: 0.011413]
2653 [D loss: 0.000074, acc.: 100.00%] [G loss: 0.009923]
2654 [D loss: 0.000070, acc.: 100.00%] [G loss: 0.009945]
2655 [D loss: 0.000103, acc.: 100.00%] [G loss: 0.011426]
2656 [D loss: 0.000081, acc.: 100.00%] [G loss: 0.008982]
2657 [D loss: 0.000082, acc.: 100.00%] [G loss: 0.009655]
2658 [D loss: 0.000072, acc.: 100.00%] [G loss: 0.009071]
2659 [D loss: 0.000078, acc.: 100.00%] [G loss: 0.008663]
2660 [D loss: 

2785 [D loss: 0.000076, acc.: 100.00%] [G loss: 0.006158]
2786 [D loss: 0.000086, acc.: 100.00%] [G loss: 0.006161]
2787 [D loss: 0.000073, acc.: 100.00%] [G loss: 0.007636]
2788 [D loss: 0.000086, acc.: 100.00%] [G loss: 0.007448]
2789 [D loss: 0.000080, acc.: 100.00%] [G loss: 0.006330]
2790 [D loss: 0.000082, acc.: 100.00%] [G loss: 0.006931]
2791 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.005450]
2792 [D loss: 0.000091, acc.: 100.00%] [G loss: 0.006960]
2793 [D loss: 0.000081, acc.: 100.00%] [G loss: 0.007036]
2794 [D loss: 0.000070, acc.: 100.00%] [G loss: 0.007557]
2795 [D loss: 0.000061, acc.: 100.00%] [G loss: 0.006764]
2796 [D loss: 0.000096, acc.: 100.00%] [G loss: 0.005648]
2797 [D loss: 0.000076, acc.: 100.00%] [G loss: 0.006753]
2798 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.005900]
2799 [D loss: 0.000067, acc.: 100.00%] [G loss: 0.005802]
2800 [D loss: 0.000069, acc.: 100.00%] [G loss: 0.005137]
2801 [D loss: 0.000077, acc.: 100.00%] [G loss: 0.005084]
2802 [D loss: 

2927 [D loss: 0.000087, acc.: 100.00%] [G loss: 0.006126]
2928 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.006408]
2929 [D loss: 0.000074, acc.: 100.00%] [G loss: 0.004704]
2930 [D loss: 0.000073, acc.: 100.00%] [G loss: 0.004556]
2931 [D loss: 0.000078, acc.: 100.00%] [G loss: 0.005027]
2932 [D loss: 0.000091, acc.: 100.00%] [G loss: 0.005279]
2933 [D loss: 0.000069, acc.: 100.00%] [G loss: 0.005306]
2934 [D loss: 0.000069, acc.: 100.00%] [G loss: 0.006722]
2935 [D loss: 0.000083, acc.: 100.00%] [G loss: 0.005882]
2936 [D loss: 0.000091, acc.: 100.00%] [G loss: 0.005477]
2937 [D loss: 0.000073, acc.: 100.00%] [G loss: 0.006597]
2938 [D loss: 0.000079, acc.: 100.00%] [G loss: 0.007449]
2939 [D loss: 0.000073, acc.: 100.00%] [G loss: 0.005742]
2940 [D loss: 0.000068, acc.: 100.00%] [G loss: 0.005762]
2941 [D loss: 0.000080, acc.: 100.00%] [G loss: 0.005408]
2942 [D loss: 0.000089, acc.: 100.00%] [G loss: 0.005872]
2943 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.004194]
2944 [D loss: 

3069 [D loss: 0.000049, acc.: 100.00%] [G loss: 0.008936]
3070 [D loss: 0.000093, acc.: 100.00%] [G loss: 0.007357]
3071 [D loss: 0.000046, acc.: 100.00%] [G loss: 0.008003]
3072 [D loss: 0.000051, acc.: 100.00%] [G loss: 0.006957]
3073 [D loss: 0.000052, acc.: 100.00%] [G loss: 0.007630]
3074 [D loss: 0.000035, acc.: 100.00%] [G loss: 0.007143]
3075 [D loss: 0.000043, acc.: 100.00%] [G loss: 0.006910]
3076 [D loss: 0.000040, acc.: 100.00%] [G loss: 0.007055]
3077 [D loss: 0.000040, acc.: 100.00%] [G loss: 0.008565]
3078 [D loss: 0.000042, acc.: 100.00%] [G loss: 0.006087]
3079 [D loss: 0.000057, acc.: 100.00%] [G loss: 0.006787]
3080 [D loss: 0.000048, acc.: 100.00%] [G loss: 0.007827]
3081 [D loss: 0.000047, acc.: 100.00%] [G loss: 0.007115]
3082 [D loss: 0.000052, acc.: 100.00%] [G loss: 0.009529]
3083 [D loss: 0.000056, acc.: 100.00%] [G loss: 0.006895]
3084 [D loss: 0.000052, acc.: 100.00%] [G loss: 0.007432]
3085 [D loss: 0.000048, acc.: 100.00%] [G loss: 0.007755]
3086 [D loss: 